In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
data = pd.read_parquet('../data/raw_data_delays.parquet')
stations_gps = gpd.read_file('../data/spatial_data/stations_gps.shp', encoding='utf-8')

In [3]:
routes = data['Relacja'].drop_duplicates()\
    .str.split(' - ')\
    .apply(lambda x: sorted(x))\
    .apply(lambda x: ','.join(map(str,x)))\
    .drop_duplicates()\
    .reset_index(drop=True)\
    .to_frame()

In [4]:
routes['Station_A'] = routes['Relacja'].str.split(',').str[0]
routes['Station_B'] = routes['Relacja'].str.split(',').str[1]

In [5]:
routes = routes\
    .merge(stations_gps, how='left', left_on='Station_A', right_on='Stacja')\
    .merge(stations_gps, how='left', left_on='Station_B', right_on='Stacja')\
    .drop(['Stacja_x','Stacja_y'], axis=1)\
    .rename(columns={'geometry_x':'geometry_A', 'geometry_y':'geometry_B'})

In [14]:
geoms = ['geometry_A','geometry_B']
suffixes = ['A', 'B']

for suffix, geom in zip(suffixes, geoms):
    print(suffix)
    print(geom)

# routes['latA'] = routes['geometry_A'].apply(lambda point: point.y)
# routes['lonA'] = routes['geometry_A'].apply(lambda point: point.x)
# routes['latB'] = routes['geometry_B'].apply(lambda point: point.y)
# routes['lonB'] = routes['geometry_B'].apply(lambda point: point.x)

A
geometry_A
B
geometry_B


In [15]:
routes.drop(columns=['geometry_A','geometry_B'],inplace=True)

In [16]:
routes.to_parquet('../data/routes_empty.parquet', index=False)

,Relacja,Station_A,Station_B,latA,lonA,latB,lonB
0,"Przeworsk,Rzeszów Główny",Przeworsk,Rzeszów Główny,50.067161,22.503232,50.043213,22.005808
1,"Lublin Główny,Zamość Wschód",Lublin Główny,Zamość Wschód,51.231485,22.568419,50.722481,23.275995
2,"Katowice,Zwardoń",Katowice,Zwardoń,50.257607,19.017146,49.504320,18.977420
3,"Ostrów Wielkopolski,Wrocław Główny",Ostrów Wielkopolski,Wrocław Główny,51.649247,17.805625,51.097968,17.037530
4,"Gryfino,Szczecin Główny",Gryfino,Szczecin Główny,53.254119,14.492962,53.418241,14.549407
...,...,...,...,...,...,...,...
709,"Wrocław Główny,Ziębice",Wrocław Główny,Ziębice,51.097968,17.037530,50.608500,17.033886
710,"Szczecin Główny,Łobez",Szczecin Główny,Łobez,53.418241,14.549407,53.636654,15.627729
711,"Warszawa Wschodnia,Łódź Kaliska",Warszawa Wschodnia,Łódź Kaliska,52.251573,21.052298,51.757822,19.430542
712,"Nowa Sól,Poznań Główny",Nowa Sól,Poznań Główny,51.798960,15.708799,52.401627,16.911579


In [17]:
routes[['latA','lonA','latB','lonB']].to_csv('../data/coordinates_to_js.csv',index=False)

,latA,lonA,latB,lonB
0,50.067161,22.503232,50.043213,22.005808
1,51.231485,22.568419,50.722481,23.275995
2,50.257607,19.017146,49.504320,18.977420
3,51.649247,17.805625,51.097968,17.037530
4,53.254119,14.492962,53.418241,14.549407
...,...,...,...,...
709,51.097968,17.037530,50.608500,17.033886
710,53.418241,14.549407,53.636654,15.627729
711,52.251573,21.052298,51.757822,19.430542
712,51.798960,15.708799,52.401627,16.911579


In [94]:
to_group = data[['Relacja','Stacja']].drop_duplicates()
grouped_df = to_group.groupby('Relacja')['Stacja'].apply(lambda x: sorted(x)).reset_index()
grouped_df['Stacja'] = grouped_df['Stacja'].apply(lambda x: ','.join(map(str, x)))

In [95]:
grouped_df['Relacja'] = grouped_df['Relacja'].str.split(' - ').apply(lambda x: sorted(x)).apply(lambda x: ','.join(map(str,x)))

In [96]:
grouped_df = grouped_df.drop_duplicates().reset_index(drop=True)

In [97]:
grouped_df = grouped_df[['Relacja']].drop_duplicates().reset_index(drop=True)

In [98]:
grouped_df['Station_A'] = grouped_df['Relacja'].str.split(',').str[0]
grouped_df['Station_B'] = grouped_df['Relacja'].str.split(',').str[1]

In [103]:
grouped_df = \
pd.merge(grouped_df, stations_gps, how='left', left_on='Station_A', right_on='Stacja')\
    .merge(stations_gps, how='left', left_on='Station_B', right_on='Stacja')\
    .drop(['Stacja_x','Stacja_y'], axis=1)\
    .rename(columns={'geometry_x':'geometry_A', 'geometry_y':'geometry_B'})

In [104]:
grouped_df

,Relacja,Station_A,Station_B,geometry_A,geometry_B
0,"Augustów,Białystok",Augustów,Białystok,POINT (23.02645 53.85254),POINT (23.13538 53.13364)
1,"Białogard,Kołobrzeg",Białogard,Kołobrzeg,POINT (15.97765 54.00999),POINT (15.57090 54.18219)
2,"Białystok,Czeremcha",Białystok,Czeremcha,POINT (23.13538 53.13364),POINT (23.35351 52.51875)
3,"Białystok,Częstochowa Stradom",Białystok,Częstochowa Stradom,POINT (23.13538 53.13364),POINT (19.10700 50.79732)
4,"Białystok,Ełk",Białystok,Ełk,POINT (23.13538 53.13364),POINT (22.36200 53.82560)
...,...,...,...,...,...
709,"Gniezno,Środa Wielkopolska",Gniezno,Środa Wielkopolska,POINT (17.60323 52.52967),POINT (17.27419 52.21833)
710,"Szczecin Główny,Świnoujście",Szczecin Główny,Świnoujście,POINT (14.54941 53.41824),POINT (14.26673 53.90472)
711,"Kraków Główny,Żegiestów",Kraków Główny,Żegiestów,POINT (19.94780 50.06867),POINT (20.78526 49.36756)
712,"Siedlce,Żyrardów",Siedlce,Żyrardów,POINT (22.27157 52.16196),POINT (20.44898 52.05247)
